In [1]:
import torch
from brt._C.router import (
    generate_indices_and_loads,
    dispatch_with_indices_and_loads,
    split_fused_cells_to_paths,
    fuse_split_cells_from_paths,
)

path_num = 4
gates = torch.randn((8, path_num)).cuda()
topk_indices = torch.topk(gates, k=1, dim=1).indices

hot_mask = (
    torch.zeros(gates.size(0), path_num, dtype=torch.int32, device=gates.device,)
    .scatter_(1, topk_indices, 1)
    .cuda()
)

supported_capacities = torch.Tensor([1, 2, 4, 8, 16]).int().cuda()
# supported_capacities = None

route_indices, dst_loads = generate_indices_and_loads(
    hot_mask, supported_capacities, capacity_padding=True, is_dst_index=True
)
print(route_indices)
print(dst_loads)


/home/whcui/.pyenv/versions/miniconda3-3.8-4.11.0/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 0, 0, 2],
        [2, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 3],
        [0, 0, 0, 4],
        [3, 0, 0, 0]], device='cuda:0', dtype=torch.int32)
tensor([4, 1, 0, 4], device='cuda:0', dtype=torch.int32)


In [2]:
in_data = torch.randn((8, path_num)).cuda()
print(in_data)
print(route_indices)
print(dst_loads)
# (out_data_1,) = dispatch_with_indices_and_loads(in_data, route_indices, dst_loads)
# print(out_data_1)
# split_data = split_fused_cells_to_paths(out_data_1, dst_loads)
# print(split_data)

out_data_1, tags = dispatch_with_indices_and_loads(
    in_data, route_indices, dst_loads, tag_generating=True
)
print(tags)
split_data, split_loads, split_tags = split_fused_cells_to_paths(
    out_data_1, dst_loads, is_load_split=True, is_tag_split=True, tags=tags
)
print(split_tags)

fused_data, new_tags, global_dst_indices = fuse_split_cells_from_paths(
    split_data, is_tag_fuse=True, tags=split_tags
)

print(fused_data)
print(new_tags)
global_dst_indices = global_dst_indices.expand_as(fused_data)
print(global_dst_indices)
# results = (
#     torch.zeros((9, 4), dtype=torch.float32)
#     .cuda()
#     .scatter_reduce(0, global_dst_indices, fused_data, reduce="sum")
# )

# out_data_1, tags = dispatch_with_indices_and_loads(
#     in_data, route_indices, dst_loads, max_path_padding=True, tag_generating=True
# )
# split_data = split_fused_cells_to_paths(out_data_1, dst_loads, max_path_padding=True,tags=tags)
# print(split_data)# print(out_data_1)

# out_data_2 = dispatch_with_indices_and_loads(
#     in_data, route_indices, dst_loads, is_1d_routing=False
# )
# print(out_data_2)
# torch.allclose(out_data_1[0], out_data_2)


tensor([[ 1.7467e+00, -2.6529e-05,  4.6493e-01, -7.9430e-01],
        [ 4.2112e-01, -4.4683e-01,  3.4100e-01, -4.6677e-01],
        [ 4.9132e-01,  5.8635e-02,  5.8778e-01,  1.8528e-02],
        [-1.7568e+00,  1.7797e-01,  4.6597e-01, -9.1231e-01],
        [ 3.0627e-01,  1.8742e+00,  4.4877e-01,  8.4431e-01],
        [-9.4262e-01,  2.0211e+00, -1.8118e+00,  1.5891e+00],
        [-1.0158e+00,  1.1762e+00,  8.3643e-01,  1.6531e+00],
        [-2.6587e-02, -1.0188e+00,  7.3847e-01,  3.0552e-01]], device='cuda:0')
tensor([[0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 0, 0, 2],
        [2, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 3],
        [0, 0, 0, 4],
        [3, 0, 0, 0]], device='cuda:0', dtype=torch.int32)
tensor([4, 1, 0, 4], device='cuda:0', dtype=torch.int32)
tensor([2, 4, 8, 0, 5, 1, 3, 6, 7], device='cuda:0', dtype=torch.int32)
[tensor([2, 4, 8, 0], device='cuda:0', dtype=torch.int32), tensor([5], device='cuda:0', dtype=torch.int32), tensor([], device='cuda:0', dtype=to

RuntimeError: The expanded size of the tensor (4) must match the existing size (9) at non-singleton dimension 1.  Target sizes: [9, 4].  Tensor sizes: [9]